In [2]:
import optuna
import vectorbt as vbt
from datetime import datetime, timedelta


from generator import BacktestDataGenerator
from processor import Processor

from sampling import Sampling, OptimConfig


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
import optuna
import vectorbt as vbt

class KamaOptimizer:
    def __init__(self, data):
        """
        Initialisation de l'optimiseur avec les données.
        :param data: Données d'entrée pour l'optimisation
        """
        self.data = data.copy()  # Utilise une copie pour ne pas affecter l'original
        self.processor = None

    def apply_kama(self, fast, slow):
        """
        Applique le KAMA rapide et lent sur les données.
        :param fast: La fenêtre du KAMA rapide
        :param slow: La fenêtre du KAMA lent
        """
        self.processor = Processor(self.data)
        self.processor.apply_kama(name='kama_fast', level='5m', window=fast)
        self.processor.apply_kama(name='kama_slow', level='5m', window=slow)
        self.data = self.data.dropna()

    def kama_strategy(self, fast, slow):
        """
        Applique la stratégie KAMA et retourne le portefeuille.
        :param fast: La fenêtre du KAMA rapide
        :param slow: La fenêtre du KAMA lent
        :return: Portefeuille de la stratégie
        """
        self.apply_kama(fast, slow)
        price = self.data.groupby(level='5m')['close'].last()
        kama_fast = self.data.groupby(level='5m')['kama_fast_5m'].last()
        kama_slow = self.data.groupby(level='5m')['kama_slow_5m'].last()
        
        entries = kama_fast > kama_slow
        exits = kama_fast < kama_slow
        pf = vbt.Portfolio.from_signals(price, entries, exits)
        return pf

    def objective(self, trial):
        """
        Fonction objectif pour Optuna. Utilisée pour l'optimisation des paramètres.
        :param trial: L'essai actuel d'Optuna
        :return: Le Sharpe ratio du portefeuille
        """
        fast = trial.suggest_int('fast', 2, 20)
        slow = trial.suggest_int('slow', 20, 50)
        
        pf = self.kama_strategy(fast, slow)
        return pf.sharpe_ratio()

    def optimize(self, n_trials=50):
        """
        Effectue l'optimisation avec Optuna.
        :param n_trials: Nombre d'essais d'optimisation
        :return: Meilleurs paramètres trouvés
        """
        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=n_trials)
        
        print("Best params:", study.best_params)
        return study.best_params
    
    def walk_forward(self, start, end, optim_config: OptimConfig, n_trials=20):
        """
        Effectue une optimisation walk-forward sur les splits générés par Sampling.
        :param start: date de début (datetime)
        :param end: date de fin (datetime)
        :param optim_config: instance d'OptimConfig
        :param n_trials: nombre d'essais Optuna par split
        :return: liste des résultats (params et Sharpe sur chaque fenêtre)
        """
        sampler = Sampling(start, end, optim_config)
        splits = sampler.generate_splits()
        results = []

        for i, ((train_start, train_end), (test_start, test_end)) in enumerate(splits):
            # Sélectionne les données pour chaque split
            train_data = self.data[(self.data.index.get_level_values('5m') >= train_start) & (self.data.index.get_level_values('5m') <= train_end)]
            test_data = self.data[(self.data.index.get_level_values('5m') >= test_start) & (self.data.index.get_level_values('5m') <= test_end)]

            # Optimisation sur train
            self.data = train_data.copy()
            study = optuna.create_study(direction='maximize')
            study.optimize(self.objective, n_trials=n_trials)
            best_params = study.best_params

            # Test sur test
            self.data = test_data.copy()
            pf = self.kama_strategy(best_params['fast'], best_params['slow'])
            sharpe = pf.sharpe_ratio()

            results.append({
                'split': i,
                'train_period': (train_start, train_end),
                'test_period': (test_start, test_end),
                'best_params': best_params,
                'test_sharpe': sharpe
            })
            print(f"Split {i}: Sharpe={sharpe:.2f}, Params={best_params}")

        return results



In [ ]:
SIZE = 0
INTERVAL = '5m'
START = datetime(2025, 1, 1)
END = datetime(2025, 3, 5)
generator = BacktestDataGenerator(interval=INTERVAL, size=SIZE,
                                  start_date=START, end_date=END)


data = generator.get_data("BTCUSDT")


optim_config = OptimConfig(
    optimization_period=30,
    gap_period=5,
    validation_period=10,
    n_splits=3
)


# Exemple d'utilisation
optimizer = KamaOptimizer(data)  # Données à fournir ici
best_params = optimizer.optimize(n_trials=5)

2025-04-26 13:02:23,555 - utils - INFO - DataGenerator initialized ✅
2025-04-26 13:02:37,220 - utils - INFO - Data loaded from DB for BTCUSDT: 18144 bars ✅
[I 2025-04-26 13:02:42,846] A new study created in memory with name: no-name-79eac9fd-c5e4-464e-ade4-e16aeacbe6a1
[I 2025-04-26 13:03:17,191] Trial 0 finished with value: 1.3777147066323858 and parameters: {'fast': 13, 'slow': 34}. Best is trial 0 with value: 1.3777147066323858.
[I 2025-04-26 13:03:17,296] Trial 1 finished with value: 0.5422296157499975 and parameters: {'fast': 13, 'slow': 22}. Best is trial 0 with value: 1.3777147066323858.
[I 2025-04-26 13:03:17,395] Trial 2 finished with value: -0.17301904084844386 and parameters: {'fast': 4, 'slow': 48}. Best is trial 0 with value: 1.3777147066323858.
[I 2025-04-26 13:03:17,482] Trial 3 finished with value: -2.020939643272845 and parameters: {'fast': 5, 'slow': 23}. Best is trial 0 with value: 1.3777147066323858.
[I 2025-04-26 13:03:17,563] Trial 4 finished with value: 1.3667621

Best params: {'fast': 13, 'slow': 34}


In [ ]:
results = optimizer.walk_forward(START, END, optim_config, n_trials=10)

[I 2025-04-26 13:03:17,925] A new study created in memory with name: no-name-a9f9f843-dddc-4500-9524-b0ad87d310cb
[I 2025-04-26 13:03:18,001] Trial 0 finished with value: 3.696650133220926 and parameters: {'fast': 12, 'slow': 29}. Best is trial 0 with value: 3.696650133220926.
[I 2025-04-26 13:03:18,059] Trial 1 finished with value: 2.016450488650532 and parameters: {'fast': 4, 'slow': 49}. Best is trial 0 with value: 3.696650133220926.
[I 2025-04-26 13:03:18,121] Trial 2 finished with value: 2.0002760613445068 and parameters: {'fast': 11, 'slow': 25}. Best is trial 0 with value: 3.696650133220926.
[I 2025-04-26 13:03:18,229] Trial 3 finished with value: 1.0320633681626543 and parameters: {'fast': 14, 'slow': 25}. Best is trial 0 with value: 3.696650133220926.
[I 2025-04-26 13:03:18,343] Trial 4 finished with value: 3.174515324342958 and parameters: {'fast': 11, 'slow': 42}. Best is trial 0 with value: 3.696650133220926.
[I 2025-04-26 13:03:18,430] Trial 5 finished with value: 2.517918

Split 0: Sharpe=-0.10, Params={'fast': 11, 'slow': 32}


[I 2025-04-26 13:03:19,003] Trial 2 finished with value: 3.862923073524103 and parameters: {'fast': 14, 'slow': 39}. Best is trial 0 with value: 4.399006453175445.
[I 2025-04-26 13:03:19,077] Trial 3 finished with value: -0.12662538746360685 and parameters: {'fast': 4, 'slow': 47}. Best is trial 0 with value: 4.399006453175445.
[I 2025-04-26 13:03:19,209] Trial 4 finished with value: -5.044666093200579 and parameters: {'fast': 16, 'slow': 20}. Best is trial 0 with value: 4.399006453175445.
[I 2025-04-26 13:03:19,323] Trial 5 finished with value: 1.1443807526399916 and parameters: {'fast': 14, 'slow': 33}. Best is trial 0 with value: 4.399006453175445.
[I 2025-04-26 13:03:19,407] Trial 6 finished with value: 8.246086190484345 and parameters: {'fast': 19, 'slow': 42}. Best is trial 6 with value: 8.246086190484345.
[I 2025-04-26 13:03:19,483] Trial 7 finished with value: 2.401807324159428 and parameters: {'fast': 8, 'slow': 38}. Best is trial 6 with value: 8.246086190484345.
[I 2025-04-26

Split 1: Sharpe=-4.30, Params={'fast': 9, 'slow': 25}


[I 2025-04-26 13:03:19,925] Trial 4 finished with value: -12.34598035201686 and parameters: {'fast': 3, 'slow': 30}. Best is trial 3 with value: 4.198043226328482.
[I 2025-04-26 13:03:20,665] Trial 5 finished with value: inf and parameters: {'fast': 5, 'slow': 31}. Best is trial 5 with value: inf.
[I 2025-04-26 13:03:20,732] Trial 6 finished with value: inf and parameters: {'fast': 16, 'slow': 23}. Best is trial 5 with value: inf.
[I 2025-04-26 13:03:20,797] Trial 7 finished with value: -114.62983904725681 and parameters: {'fast': 13, 'slow': 39}. Best is trial 5 with value: inf.
[W 2025-04-26 13:03:20,842] Trial 8 failed with parameters: {'fast': 20, 'slow': 45} because of the following error: ValueError('Index frequency is None. Pass it as `freq` or define it globally under `settings.array_wrapper`.').
Traceback (most recent call last):
  File "c:\Users\Dell\miniconda3\envs\csyst_test\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(tri

ValueError: Index frequency is None. Pass it as `freq` or define it globally under `settings.array_wrapper`.

In [ ]:
e e 

In [ ]:
# Charger les prix BTC-USD
price = data['close']

# Définir deux moyennes mobiles
fast_ma = price.rolling(window=20).mean()
slow_ma = price.rolling(window=50).mean()

# Générer signaux
entries = fast_ma > slow_ma  # On entre en position quand rapide > lente
exits = fast_ma < slow_ma    # On sort quand rapide < lente

# Créer un portefeuille à partir des signaux
portfolio = vbt.Portfolio.from_signals(
    close=price,
    entries=entries,
    exits=exits,
    init_cash=10_000,
    fees=0.001  # exemple de frais 0.1%
)


In [ ]:
# Résultats
print(portfolio.stats())
portfolio.plot().show()
